In [112]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson

## 1 DW-тест
### 1.2 Consumption equation
Для набора данных `Consumption` рассморим регессию с серийной коррецяией первого порядка

Модель с серийной корреляцией первого порядка

Δlog(ydt)=β0+β1Δlog(cet)+ut

ut=ρut−1+vt

In [113]:
source_file = 'Consumption.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,Unnamed: 0,yd,ce
0,1,59505.0,57168
1,2,59717.4,55464
2,3,59039.1,56332
3,4,61342.6,55836
4,5,60544.6,54488
...,...,...,...
195,196,390576.6,359372
196,197,391381.5,363896
197,198,389502.6,364428
198,199,388282.1,366224


In [114]:
df['diff_log_yd'] = pd.Series(np.diff(np.log(df['yd'])))
df['diff_log_ce'] = pd.Series(np.diff(np.log(df['ce'])))
df

,Unnamed: 0,yd,ce,diff_log_yd,diff_log_ce
0,1,59505.0,57168,0.003563,-0.030260
1,2,59717.4,55464,-0.011423,0.015529
2,3,59039.1,56332,0.038275,-0.008844
3,4,61342.6,55836,-0.013094,-0.024438
4,5,60544.6,54488,-0.003640,-0.015015
...,...,...,...,...,...
195,196,390576.6,359372,0.002059,0.012510
196,197,391381.5,363896,-0.004812,0.001461
197,198,389502.6,364428,-0.003138,0.004916
198,199,388282.1,366224,0.001343,0.013593


In [115]:
# model = smf.ols(data=df, formula='diff_log_yd ~ diff_log_ce').fit()
# model = smf.ols(data=df, formula='np.log(yd) ~ np.log(ce)').fit()
model = smf.ols(data=df, formula='pd.Series(np.diff(np.log(yd))) ~ pd.Series(np.diff(np.log(ce)))').fit()
# model.params.round(2)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            diff_log_yd   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     30.06
Date:                Mon, 08 May 2023   Prob (F-statistic):           1.27e-07
Time:                        15:21:52   Log-Likelihood:                 579.44
No. Observations:                 199   AIC:                            -1155.
Df Residuals:                     197   BIC:                            -1148.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0059      0.001      5.252      0.000       0.004       0.008
diff_log_ce     0.3706      0.068      5.483      0.000       0.237       0.504
==============================================================================
Omnibus:                        0.730   Durbin-Watson:                   2.382
Prob(Omnibus):                  0.694   Jarque-Bera (JB):                0.803
Skew:                           0.140   Prob(JB):                        0.669
Kurtosis:                       2.866   Cond. No.                         72.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Способ 1
Вычисляем результат DW-теста по формуле.

$$
DW = \frac{\sum\limits_{t=2}^{n}(e_t-e_{t-1})^2}{\sum\limits_{t=1}^{n}e_t^2}
$$

In [116]:
# вычислим знаменатель
resSq = sum(model.resid**2)

In [117]:
# Вычислим числитель
sum_resid = 0
for i in range(len(model.resid)-1):
    sum_resid+=(model.resid[i+1]-model.resid[i])**2

In [118]:
DW = sum_resid/resSq
print(f'DW = {DW:.3f}')

DW = 2.382


### Способ 2
Вычисляем результат DW-теста с помощью функции библиотеки statsmodels.

In [119]:
DW = durbin_watson(model.resid)
print(f'DW = {DW:.3f}')

DW = 2.382


In [120]:
reply = [
    'Гипотеза подтверждается, автокорреляции нет DW == 0',
    'Гипотеза отвергается, есть серийная корреляция DW <> 0'
] 
print(reply[0] if DW == 0 else reply[1])

Гипотеза отвергается, есть серийная корреляция DW <> 0


In [121]:
j = 4 # Кол-вл регрессоров
alpha = 0.05 # Уровень значимости
f_cr = stats.f.ppf(1-alpha,j,len(df)-j).round(2)
print(f'F_cr = {f_cr:.2f}')

F_cr = 2.42
